In [1]:
import torch
from torch import nn

In [5]:
net = nn.Sequential(nn.Conv2d(1,6, kernel_size=5, padding=2), nn.Sigmoid(),
                    nn.AvgPool2d(kernel_size=2,stride=2),
                    nn.Conv2d(6,16,kernel_size=5), nn.Sigmoid(),
                    nn.AvgPool2d(kernel_size=2, stride=2),
                    nn.Flatten(),
                    nn.Linear(16 * 5 * 5, 120), nn.Sigmoid(),
                    nn.Linear(120,84), nn.Sigmoid(), nn.Linear(84,10)
                    )

In [6]:
X = torch.randn((1,1,28,28))

In [7]:
for layer in net:
    X = layer(X)`
    print(f"Shape of {layer.__class__.__name__} : {X.shape}")

Shape of Conv2d : torch.Size([1, 6, 28, 28])
Shape of Sigmoid : torch.Size([1, 6, 28, 28])
Shape of AvgPool2d : torch.Size([1, 6, 14, 14])
Shape of Conv2d : torch.Size([1, 16, 10, 10])
Shape of Sigmoid : torch.Size([1, 16, 10, 10])
Shape of AvgPool2d : torch.Size([1, 16, 5, 5])
Shape of Flatten : torch.Size([1, 400])
Shape of Linear : torch.Size([1, 120])
Shape of Sigmoid : torch.Size([1, 120])
Shape of Linear : torch.Size([1, 84])
Shape of Sigmoid : torch.Size([1, 84])
Shape of Linear : torch.Size([1, 10])


In [11]:
batch_size = 256


import torchvision.datasets as datasets
import torchvision.transforms as transforms

In [13]:
my_transforms = transforms.ToTensor()

train_dataset = datasets.FashionMNIST(root="../data", download=False, train=True, transform=my_transforms)
test_dataset = datasets.FashionMNIST(root="../data", download=False, train=False, transform=my_transforms)

In [16]:
X,y = train_dataset[0]
X.shape,y

(torch.Size([1, 28, 28]), 9)

In [17]:
train_dataloader = torch.utils.data.DataLoader(train_dataset, shuffle=True, batch_size=batch_size, num_workers=4)
test_dataloader = torch.utils.data.DataLoader(test_dataset, shuffle=False, batch_size=batch_size, num_workers=4)

Since the full dataset is in the main memory, we need to copy it to the
GPU memory before the model uses GPU to compute with the dataset

In [18]:
def accuracy(y_hat, y):
    return (y_hat.argmax(1) == y).sum()

In [19]:
def evaluate_accuracy_gpu(net, data_iter, device=None):
    if isinstance(net, nn.Module):
        net.eval()
        if not device:
            device = next(iter(net.parameters())).device
    
    metric_0 = []
    metric_1 = []
    
    with torch.no_grad():
        for X, y in data_iter:
            if isinstance(X, list):
                X = [x.to(device) for x in X]
            else :
                X = X.to(device)
                
        y = y.to(device)
        metric_0.append(accuracy(net(X), y))
        metric_1.append(accuracy(y.numel()))
    return metric_0/metric_1

In [28]:
def train_ch6(net, train_iter, test_iter, num_epochs, lr, device):
    def init_weights(m):
        if(type(m)==nn.Linear or type(m) == nn.Conv2d):
            nn.init.xavier_uniform_(m.weight)
        
    net.apply(init_weights)
    print('training on', device)
    net.to(device)
    
    optimizer = torch.optim.SGD(net.parameters(),lr)
    loss = nn.CrossEntropyLoss()
    
    train_l = []
    train_acc = []
    test_acc = []
    
    num_batches = len(train_iter)
    for epoch in range(num_epochs):
        net.train()
        metric_0 = []
        metric_1 = []
        metric_2 = []
        for i, (X, y) in enumerate(train_iter):
            X, y = X.to(device), y.to(device)
            y_hat = net(X)
            l = loss(y_hat, y)
            optimizer.zero_grad()
            l.backward()
            optimizer.step()
            
            metric_0.append(l * X.shape[0])
            metric_1.append(accuracy(y_hat, y))
            metric_2.append(X.shape[0])
            
            train_l.append(metric_1[-1] / metric_2[-1])
            train_acc.append(metric_1[-1] / metric_2[-1])
            
            if (i + 1) % (num_batches // 5) == 0 or i == num_batches - 1:
                print(f"the train_l for current batch {train_l[-1]}, train accuracy {train_acc[-1]}")
            test_acc.append(evaluate_accuracy_gpu(net, test_iter))
    print(f'loss {train_l[-1]:.3f}, train acc {train_acc[-1]:.3f}, '
    f'test acc {test_acc[-1]:.3f}')
            

In [29]:
lr, num_epochs = 0.9, 10

device = torch.device('cuda')

train_ch6(net, train_iter=train_dataloader, test_iter=test_dataloader, num_epochs=num_epochs, lr=0.9, device=device)

training on cuda


RuntimeError: DataLoader worker (pid(s) 14284, 5564, 16432, 2556) exited unexpectedly